In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import numpy as np
from kaggle_datasets import KaggleDatasets

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

Load and visualize the data

from experimentation I found image size of 256 worked best

In [2]:
img_height = 256
img_width = 256

GCS_PATH = KaggleDatasets().get_gcs_path()

train_path = "../input/chest-xray-pneumonia/chest_xray/train"
test_path = "../input/chest-xray-pneumonia/chest_xray/test"
valid_path = "../input/chest-xray-pneumonia/chest_xray/val"

CATEGORIES = ["NORMAL", "PNEUMONIA"]

for category in CATEGORIES:
	path = os.path.join(train_path, category)
	for img in os.listdir(path):
		img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
		new_array = cv2.resize(img_array, (img_width, img_height))
		plt.imshow(new_array, cmap='gray')
		plt.show()
		break
	break

Data spliting

It is important that the data is split properly because there are only 7 images in the validation file

I just dragged and dropped to get an 80:20 split. I couldn't do that here so I'll have a lower validation accuracy and a high validation loss.

Data Augmentation

I found the values for data augmentation were the best.

In [3]:
shear_range = 0.2
zoom_range = 0.3
rotation_range = 0
horizontal_flip = False

def img_augmentation(shear_range, zoom_range, rotation_range, horizontal_flip):
	global train, test, valid
	image_gen = ImageDataGenerator(rescale=1/255,
			shear_range=shear_range,
			zoom_range=zoom_range,
			rotation_range=rotation_range,
			horizontal_flip=horizontal_flip)

	test_data_gen = ImageDataGenerator(rescale=1/255)

	train = image_gen.flow_from_directory(
		train_path,
		target_size=(img_height, img_width),
		shuffle=True,
		color_mode='grayscale',
		batch_size=batch_size,
		class_mode='binary',
		)

	test = test_data_gen.flow_from_directory(
		test_path,
		target_size=(img_height, img_width),
		shuffle=False,
		color_mode='grayscale',
		class_mode='binary',
		batch_size=64
		)

	valid = image_gen.flow_from_directory(
		valid_path,
		target_size=(img_height, img_width),
		color_mode='grayscale',
		shuffle=False,
		class_mode='binary',
		batch_size=64
		)

Model building

I expirmented with many different model structures and came across a model with low validation loss

In [4]:
class build_model(object):
	def __init__(self, activation, conv_layer, dense_layer, layer_size):
		self.activation = activation
		self.conv_layer = conv_layer
		self.dense_layer = dense_layer
		self.layer_size = layer_size

	def conv_block(self, filter_size):
		block = [Conv2D(filter_size, (3, 3), activation=self.activation, input_shape=(img_width, img_height, 1), padding='same'),
		MaxPooling2D(pool_size=(2, 2))]

		l = 1
		while l < self.conv_layer:
			block.append(Conv2D(filter_size*(2**l), (3, 3), activation=self.activation, padding='same')),
			block.append(MaxPooling2D(pool_size=(2, 2)))
			block.append(Dropout(rate=0.2))

			l = l + 1

		return block

	def dense_block(self):
		block=[Dense(activation=self.activation, units=self.layer_size)]

		l = 2
		while l < self.dense_layer+1:
			block.append(Dense(activation=self.activation, units=self.layer_size/l))
			l = l + 1

			return block

I put the model all together in a sequential function

In [5]:
def model_add(filter_size, activation, dropout_rate):
	global model

	CONV_LAYER = 3
	LAYER_SIZE = 128
	DENSE_LAYER = 2

	NN = build_model(activation, CONV_LAYER, DENSE_LAYER, LAYER_SIZE)

	model = Sequential([
		NN.conv_block(filter_size)[0],
		NN.conv_block(filter_size)[1],
		NN.conv_block(filter_size)[2],
		NN.conv_block(filter_size)[3],
		NN.conv_block(filter_size)[4],
		NN.conv_block(filter_size)[5],
		NN.conv_block(filter_size)[6],
		NN.conv_block(filter_size)[7],
		Flatten(),
		NN.dense_block()[0],
		NN.dense_block()[1],
		Dropout(rate=0.2),
		Dense(activation='sigmoid', units=1)])

	return model

In [ ]:
then I make my callbacks list and fit the model to my training data

In [6]:
batch_size = 64

class model_eval(object):
	def __init__(self, train, test, val, epochs):
		self.train = train
		self.test = test
		self.val = val
		self.epochs = epochs

	def balance_classes(self):
		weights = compute_class_weight(class_weight='balanced', classes=np.unique(train.classes), y=train.classes)
		cw = dict(zip(np.unique(train.classes), weights))

		return cw


	def callback_list(self):
		early = EarlyStopping(monitor='val_accuracy', patience=5)

		learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
			patience=1, verbose=1, factor=0.5, min_lr=0.000001)

		checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("model.h5", 
			save_best_only=True)

		callback_list = [early, learning_rate_reduction]

		return callback_list


	def train_test(self, optimizer):
		global history
		model_add(32, 'relu', 0.2)
		model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
		model.summary()

		cw = self.balance_classes()
		callbacks_list = self.callback_list()

		history = model.fit(self.train, epochs=self.epochs, validation_data=self.val, class_weight=cw, callbacks=callbacks_list)

		test_accuracy = model.evaluate(self.test)
		print('The testing accuracy is: ', test_accuracy[1]*100, '%')

		return history

In [ ]:
Then I graph my validation accuracy and validation loss


In [8]:
def main():
	img_augmentation(shear_range, zoom_range, rotation_range, horizontal_flip)
	epochs = 50

	eval = model_eval(train, test, valid, epochs)
	eval.train_test('adam')

	accuracy = history.history['accuracy']
	val_accuracy = history.history['val_accuracy']
	plt.plot(range(len(accuracy)), accuracy, color='blue', label='Training accuracy')
	plt.plot(range(len(accuracy)), val_accuracy, color='red', label='Validation accuracy')
	plt.xlabel('Epoch No.')
	plt.ylabel('Accuracy')
	plt.legend()
	plt.show()

	loss = history.history['loss']
	val_loss = history.history['val_loss']
	plt.plot(range(len(accuracy)), loss, color='blue', label='Training loss')
	plt.plot(range(len(accuracy)), val_loss, color='red', label='Validation loss')
	plt.xlabel('Epoch No.')
	plt.ylabel('loss')
	plt.legend()
	plt.show()

	true = test.classes
	preds = model.predict(test, verbose=1)
	predictions = preds.copy()
	predictions[predictions <= 0.5] = 0
	predictions[predictions > 0.5] = 1

	cm = confusion_matrix(true, np.round(predictions))
	fig, ax = plt.subplots()
	fig.set_size_inches(12, 8)
	sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap=plt.cm.Blues, cbar=False)
	ax.tick_params(axis='x', labelsize=16)
	ax.tick_params(axis='y', labelsize=16)
	ax.set_ylabel("True", color="royalblue", fontsize=35, fontweight=700)
	ax.set_xlabel("Prediction", color="royalblue", fontsize=35, fontweight=700)
	plt.yticks(rotation=0)
	plt.show()

	print(classification_report(y_true=test.classes, y_pred=predictions, target_names=['NORMAL', 'PNEUMONIA']))

if __name__ == '__main__':
	main()